**Train Test and Cross Validation**

In [1]:
import pandas as pd
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score


In [2]:
# Load training data
data_path = '/workspaces/Final-Year-Project/Cleaned Data/TrainTestData.csv'
df = pd.read_csv(data_path)

# Drop missing values
df = df.dropna()

# Features and target variable
X = df.drop('Diabetes Status', axis=1)
y = df['Diabetes Status']


In [3]:
# Feature scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Display the shape after scaling
print(f"📊 NB - Final dataset shape after scaling: {X_scaled.shape}")


📊 NB - Final dataset shape after scaling: (3268, 13)


In [4]:
# Gaussian Naive Bayes Classifier setup
nb = GaussianNB()

# Hyperparameter grid for GridSearchCV
param_grid = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4]  # Vary smoothing for more stability
}

# KFold cross-validation setup
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# GridSearchCV setup with accuracy as the scoring parameter
grid_search = GridSearchCV(nb, param_grid, scoring='accuracy', cv=kf, n_jobs=-1, verbose=1)


In [5]:
# Fit GridSearchCV on the training data
grid_search.fit(X_scaled, y)

# Best model from GridSearchCV
best_nb = grid_search.best_estimator_

# Print the best hyperparameters and the best score
print("Best Parameters for Naive Bayes:", grid_search.best_params_)
print("Best Accuracy with Best Parameters:", grid_search.best_score_)


Fitting 5 folds for each of 6 candidates, totalling 30 fits


Best Parameters for Naive Bayes: {'var_smoothing': 1e-09}
Best Accuracy with Best Parameters: 0.7255274409804665


In [6]:
# Predictions on training data
y_pred = best_nb.predict(X_scaled)

# Classification report on the training data
print("📌 NB Classification Report on Training Data:")
print(classification_report(y, y_pred))

# Confusion matrix on the training data
print("🟦 NB - Confusion Matrix on Training Data:")
print(confusion_matrix(y, y_pred))


📌 NB Classification Report on Training Data:
              precision    recall  f1-score   support

           0       0.78      0.64      0.70      1638
           1       0.69      0.81      0.75      1630

    accuracy                           0.73      3268
   macro avg       0.73      0.73      0.73      3268
weighted avg       0.74      0.73      0.73      3268

🟦 NB - Confusion Matrix on Training Data:
[[1056  582]
 [ 306 1324]]


In [7]:
# Load validation data
valid_path = "/workspaces/Final-Year-Project/Cleaned Data/ValidationData.csv"
df_valid = pd.read_csv(valid_path)

# Drop any rows with missing values in the validation set
df_valid = df_valid.dropna()

# Prepare features and target for validation data
X_valid = df_valid.drop('Diabetes Status', axis=1)
y_valid = df_valid['Diabetes Status']

# Scale the validation data using the previously fitted scaler
X_valid_scaled = scaler.transform(X_valid)

# Display the shape of the validation dataset
print(f"🧪 NB - Validation dataset shape: {X_valid.shape}")


🧪 NB - Validation dataset shape: (364, 13)


In [8]:
# Predictions on validation data
y_pred_valid = best_nb.predict(X_valid_scaled)


In [9]:
# Print classification report for validation data
print(f"📌 NB - Validation Classification Report:")
print(classification_report(y_valid, y_pred_valid))

# Print confusion matrix for validation data
print("🟦 NB - Validation Confusion Matrix:")
print(confusion_matrix(y_valid, y_pred_valid))


📌 NB - Validation Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.66      0.72       178
           1       0.72      0.85      0.78       186

    accuracy                           0.76       364
   macro avg       0.76      0.75      0.75       364
weighted avg       0.76      0.76      0.75       364

🟦 NB - Validation Confusion Matrix:
[[117  61]
 [ 28 158]]


In [10]:
import pandas as pd
from sklearn.metrics import classification_report

# Assuming you already have y_valid (true labels) and y_pred_valid (predicted labels)
# Example classification report for AdaBoost (AB)
report = classification_report(y_valid, y_pred_valid, output_dict=True)

# Extract weighted scores
weighted_precision = report['weighted avg']['precision']
weighted_recall = report['weighted avg']['recall']
weighted_f1 = report['weighted avg']['f1-score']
accuracy = report['accuracy']

# Create a dictionary with the scores
ab_results = {
    'Model': 'Naive Bayes(LR)',
    'Precision': weighted_precision,
    'Recall': weighted_recall,
    'F1-Score': weighted_f1,
    'Accuracy': accuracy
}

# Convert the dictionary to a DataFrame
ab_df = pd.DataFrame([ab_results])

# Create a DataFrame to store results (if not already created)
results_df = pd.read_csv("/workspaces/Final-Year-Project/Results/model_results.csv")

# Concatenate the new row to the DataFrame
results_df = pd.concat([results_df, ab_df], ignore_index=True)

# Print the DataFrame to confirm it's added
print(results_df)

# Save the DataFrame to a CSV file
results_df.to_csv("/workspaces/Final-Year-Project/Results/model_results.csv", index=False)


                                Model  Precision    Recall  F1-Score  Accuracy
0                       AdaBoost (AB)   0.767212  0.752747  0.748389  0.752747
1                  Neural Network(NN)   0.762241  0.755495  0.753238  0.755495
2                   Decision Tree(DT)   0.748498  0.741758  0.739233  0.741758
3   K Nearest Nearest Neighbours(KNN)   0.723548  0.722527  0.721802  0.722527
4                  Neural Network(NN)   0.775001  0.769231  0.767458  0.769231
5                   Decision Tree(DT)   0.748498  0.741758  0.739233  0.741758
6   K Nearest Nearest Neighbours(KNN)   0.723548  0.722527  0.721802  0.722527
7            Logistic Regression (LR)   0.759684  0.755495  0.753959  0.755495
8            Logistic Regression (LR)   0.759684  0.755495  0.753959  0.755495
9            Logistic Regression (LR)   0.759684  0.755495  0.753959  0.755495
10                  Random Forest(RF)   0.744664  0.741758  0.740463  0.741758
11                  Random Forest(RF)   0.748448  0.

In [11]:
# Recalculate Macro average F1-score on validation data
validation_f1_macro = f1_score(y_valid, y_pred_valid, average='macro')
print(f"🏆 Best Model Validation F1-Score: {validation_f1_macro}")


🏆 Best Model Validation F1-Score: 0.7523525589573061


In [14]:
import matplotlib.pyplot as plt

# Group means by diabetes status
means_by_class = X_scaled.groupby('Diabetes Status').mean()

# Compute mean differences (positive class - negative class)
mean_diff = means_by_class.loc[1] - means_by_class.loc[0]
mean_diff = mean_diff.sort_values(ascending=False)

# Plot inline
plt.figure(figsize=(10, 6))
mean_diff.plot(kind='bar')
plt.title("Feature Influence in Naive Bayes: Class Mean Differences (Standardized)")
plt.ylabel("Mean Difference (Diabetic - Non-Diabetic)")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

AttributeError: 'numpy.ndarray' object has no attribute 'groupby'